## Dynamic Plans

Here, instead of a predefined plan, vehicles take decisions online following a minimization distance rule. Only a goal is predefined.

In [1]:
import numpy as np


# Custom modules
from helpers import global2local, plot_3d_interactive, kill_processes
from helpers.change_coordinates import global2local_broadcast
from simulators import Simulator, Gazebo, QGC,Color
from plan import Plan, State,PlanMode
from oracle import Oracle


from helpers.navegation_logic import find_path, find_best_waypoint


kill_processes()

## Gonfiguration

In [2]:
offsets = [ #east, north, up, heading
    (40, -10, 0, 0), 
    (40,  20, 0, 0)]
n_vehicles = len(offsets)

wps = np.array([(40, -10, 5),
                (15, -10, 5),
                (0, -10, 5),
                (0,   0, 5),
                (0,   10, 5),
                (40, 20, 5),
                (15, 20, 5),
                (0,  20, 5)])

homes=np.array([offset[:3] for offset in offsets])
target_wps=np.array([
                (0, 10,5),
                (0, 0,5)])
local_target_wps=global2local(target_wps, homes)
local_wps=global2local_broadcast(wps, homes)

targets=np.array([(*target_wp[:2],0) for target_wp in target_wps])


plans=[Plan.basic(wps=local_target_wps[0][np.newaxis,:],navegation_speed=1.5, mode=PlanMode.DYNAMIC,wp_margin=0.5,dynamic_wps=local_wps[0]),
       Plan.basic(wps=local_target_wps[1][np.newaxis,:],navegation_speed=1.5,mode=PlanMode.DYNAMIC,wp_margin=0.5,dynamic_wps=local_wps[1]),
       ]

## Visualization Parameters

In [3]:
markers = {'waypoints': {'pos':wps,'color':Color.YELLOW},
           'home1': {'pos':homes[[0]],'color':Color.BLUE},
           'home2': {'pos':homes[[1]],'color':Color.RED},
           'target1':{'pos':targets[[0]],'color':Color.BLUE},
           'target2':{'pos':targets[[1]],'color':Color.RED}}
plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.4],ground=-0.05)

## Waypoint selection algorithm

This is for checking the decisions panl that will be taken online for checking propouses. The path is shown in local coordinates

In [4]:
for i,(target_wp,wps) in enumerate(zip(local_target_wps,local_wps)):
        local_path=find_path(np.array((0,0,0)),target_wp, wps,eps=1)
        print(f'uav-{i} Best path: {local_path}') 


uav-0 Best path: [[  0   0   0]
 [  0   0   5]
 [-25   0   5]
 [-40   0   5]
 [-40  10   5]
 [-40  20   5]]
uav-1 Best path: [[  0   0   0]
 [  0   0   5]
 [-25   0   5]
 [-40   0   5]
 [-40 -20   5]]


## Choose Simulator

In [5]:
#simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path="simulators/gazebo/worlds/small_city.world",
                 models=n_vehicles*["iris"],
                 colors=[Color.BLUE,Color.RED],
                 markers=markers)

Launch Simulator

In [6]:
uavs = simulator.launch()
orac = Oracle(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀


## Execute Plan 

This should be refactored and add a dynamic plan propety in plan or a dynamic_mission uav mode

In [7]:
while orac.vehs:
    orac.gather_broadcasts()
    for uav in orac.vehs.copy():
        orac.update_neighbors(uav)
        if uav.plan.state == State.DONE:
            orac.remove(uav)
        else:
            uav.act()
kill_processes()

Vehicle 1: ▶️ Plan Started: 📋 basic
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 2: ▶️ Plan Started: 📋 basic
Vehicle 2: ▶️ Action Started: 🔧 PREARM
Vehicle 2: ▶️ Step Started: Check disarmed
Vehicle 2: ✅ Step Done: Check disarmed
Vehicle 2: ▶️ Step Started: Check EKF
Vehicle 1: ✅ Step Done: Check disarmed
Vehicle 1: ▶️ Step Started: Check EKF
Vehicle 2: ✅ Step Done: Check EKF
Vehicle 2: ▶️ Step Started: Check GPS
Vehicle 1: ✅ Step Done: Check EKF
Vehicle 1: ▶️ Step Started: Check GPS
Vehicle 2: ✅ Step Done: Check GPS
Vehicle 2: ▶️ Step Started: Check system
Vehicle 1: ✅ Step Done: Check GPS
Vehicle 1: ▶️ Step Started: Check system
Vehicle 2: ✅ Step Done: Check system
Vehicle 2: ✅ Action Done: 🔧 PREARM
Vehicle 2: ▶️ Action Started: ⚙️ MODE: GUIDED
Vehicle 2: ▶️ Step Started: Switch to GUIDED
Vehicle 2: ✅ Step Done: Switch to GUIDED
Vehicle 2: ✅ Action Done: ⚙️ MODE: GUIDED
Vehicle 2: ▶️ Action Started: 🎚️ CHANGE_NAV_SPEED
Vehicle 2: ▶️ Step S